# Demagnetization Potential in 3D with scikit-fem

We solve the demagnetization potential problem on a 3D mesh loaded from GMSH:
$$-\Delta \phi = \nabla \cdot \mathbf{m} \quad \text{in } \Omega$$
$$\phi = 0 \quad \text{on } \partial\Omega$$

where:
- $\phi$ is the demagnetization potential
- $\mathbf{m} = (0, 0, 1)$ is the magnetization vector (uniform in z-direction)
- The source term $\nabla \cdot \mathbf{m}$ is only active in the magnetic domain (region 1)
- The air region (region 2) has no magnetization

This models the magnetic field outside a uniformly magnetized cube.

## Step 1: Import Required Libraries

We import libraries for 3D finite element computations:
- GMSH mesh loading capabilities
- 3D tetrahedral elements
- 3D visualization tools

In [8]:
import numpy as np
from skfem import Mesh, Basis, ElementTetP1, ElementTetDG0, asm
from skfem.helpers import *
from skfem import BilinearForm, LinearForm
from skfem.utils import condense

## Step 2: Load Mesh and Define Function Space

We load the 3D mesh from the GMSH file:
- The mesh contains a cube (magnetic region, tag=1) surrounded by air (tag=2)
- We use P1 tetrahedral elements for the scalar potential
- The mesh regions allow us to apply different material properties

In [9]:
# Load mesh from GMSH file
m = Mesh.load('files/cube_with_air.msh')
e = ElementTetP1()
basis = Basis(m, e)

# Create DG0 basis for piecewise constant magnetization
e_dg0 = ElementTetDG0()
basis_dg0 = Basis(m, e_dg0)

print(f"Mesh has {m.p.shape[1]} nodes and {m.t.shape[1]} tetrahedra")
print(f"Mesh bounding box:")
print(f"  x: [{m.p[0].min():.3f}, {m.p[0].max():.3f}]")
print(f"  y: [{m.p[1].min():.3f}, {m.p[1].max():.3f}]")
print(f"  z: [{m.p[2].min():.3f}, {m.p[2].max():.3f}]")

# Check available subdomains
if hasattr(m, 'subdomains'):
    print(f"Available subdomains: {list(m.subdomains.keys())}")
else:
    print("No subdomain information found")

Failure to parse tags from meshio.



Mesh has 3665 nodes and 20679 tetrahedra
Mesh bounding box:
  x: [-5.500, 5.500]
  y: [-5.500, 5.500]
  z: [-5.500, 5.500]
Available subdomains: ['magnetic', 'air']


## Step 3: Define Weak Form (Variational Formulation)

The weak form of our problem is: Find $\phi$ such that
$$\int_{\Omega} \nabla \phi \cdot \nabla v \, d\Omega = \int_{\Omega_{mag}} \mathbf{m} \cdot \nabla v \, d\Omega \quad \forall v$$

We define:
- **Bilinear form**: $a(\phi,v) = \int \nabla \phi \cdot \nabla v \, d\Omega$ (Laplacian)
- **Linear form**: $L(v) = \int_{\Omega_{mag}} \mathbf{m} \cdot \nabla v \, d\Omega$ (magnetization source)

The magnetization $\mathbf{m} = (0, 0, 1)$ creates a source term only in the magnetic region.

In [10]:
# Define bilinear form: ∫ ∇φ · ∇v dΩ (Laplacian)
@BilinearForm
def laplace_form(u, v, _):
    return ddot(u.grad, v.grad)

# Create piecewise constant magnetization function using DG0 elements
# Full magnetization vector m = (mx, my, mz)
magnetization_x = np.zeros(basis_dg0.N)
magnetization_y = np.zeros(basis_dg0.N)
magnetization_z = np.zeros(basis_dg0.N)

# Set magnetization values based on subdomain information
if hasattr(m, 'subdomains') and 'magnetic' in m.subdomains:
    # Get elements in magnetic subdomain
    magnetic_elements = m.subdomains['magnetic']
    # Set magnetization vector m = (0, 0, 1) in magnetic domain
    magnetization_x[magnetic_elements] = 0.0
    magnetization_y[magnetic_elements] = 0.0
    magnetization_z[magnetic_elements] = 1.0
    print(f"Set magnetization m=(0,0,1) in {len(magnetic_elements)} magnetic elements")
elif hasattr(m, 'subdomains') and '1' in m.subdomains:
    # Fallback to numeric subdomain tag
    magnetic_elements = m.subdomains['1']
    magnetization_x[magnetic_elements] = 0.0
    magnetization_y[magnetic_elements] = 0.0
    magnetization_z[magnetic_elements] = 1.0
    print(f"Set magnetization m=(0,0,1) in {len(magnetic_elements)} magnetic elements (subdomain '1')")
else:
    print("Warning: No magnetic subdomain found, magnetization will be zero everywhere")

# Define linear form: ∫ m · ∇v dΩ (magnetization source)
@LinearForm
def magnetization_form(v, w):
    # Interpolate DG0 magnetization components to quadrature points
    mx = basis_dg0.interpolate(magnetization_x)(w)
    my = basis_dg0.interpolate(magnetization_y)(w)
    mz = basis_dg0.interpolate(magnetization_z)(w)
    # m · ∇v = mx * ∂v/∂x + my * ∂v/∂y + mz * ∂v/∂z
    return mx * v.grad[0] + my * v.grad[1] + mz * v.grad[2]

## Step 4: Assembly

We assemble the system matrices:
- Stiffness matrix from the Laplacian (assembled over entire domain)
- Load vector from magnetization (assembled only over magnetic region)

The key is to restrict the magnetization source to the magnetic domain only.

In [11]:
# Assemble stiffness matrix over entire domain
A = asm(laplace_form, basis)

# Assemble load vector over full domain using piecewise constant magnetization
b = asm(magnetization_form, basis)
print("Using DG0 piecewise constant magnetization over full domain")

print(f"System size: {A.shape[0]} x {A.shape[1]}")
print(f"Load vector norm: {np.linalg.norm(b):.3e}")

Using full domain assembly (fallback)
System size: 3665 x 3665
Load vector norm: 2.645e+01


## Step 5: Apply Boundary Conditions and Solve

We apply homogeneous Dirichlet boundary conditions $\phi = 0$ on all external boundaries:
- Find all boundary nodes
- Use the `condense` method for symmetric boundary condition application
- Solve the reduced system

This approach maintains the symmetric structure of the problem.

In [12]:
# Find boundary nodes (all external boundaries)
boundary = m.boundary_nodes()
print(f"Found {len(boundary)} boundary nodes")

# Use condense method for symmetric BC application
interior = basis.complement_dofs(boundary)
A_int, b_int, *_ = condense(A, b, I=interior)

print(f"Reduced system size: {A_int.shape[0]} x {A_int.shape[1]}")

# Solve reduced system
phi_int = np.linalg.solve(A_int.toarray(), b_int)

# Reconstruct full solution
phi = np.zeros(basis.N)
phi[interior] = phi_int
phi[boundary] = 0.0  # Homogeneous Dirichlet BC

print(f"Solution computed with {len(phi)} degrees of freedom")
print(f"Solution range: [{phi.min():.6f}, {phi.max():.6f}]")

Found 221 boundary nodes
Reduced system size: 3444 x 3444
Solution computed with 3665 degrees of freedom
Solution range: [-0.000000, 0.000000]


## Step 6: Export to VTU File

We export the 3D demagnetization potential solution to a VTU file for visualization in ParaView or other VTK-compatible software:
- The mesh geometry and connectivity are preserved
- The potential field is saved as point data
- The magnetic field (gradient of potential) can be computed in post-processing

This allows for advanced 3D visualization and analysis of the magnetic field distribution.

In [13]:
# Export solution to VTU file
output_filename = 'demag_potential_3d.vtu'

# Create a dictionary with the solution data
point_data = {
    'demagnetization_potential': phi
}

# Save to VTU format
m.save(output_filename, point_data=point_data)

print(f"Solution exported to {output_filename}")
print(f"File contains:")
print(f"  - Mesh with {m.p.shape[1]} nodes and {m.t.shape[1]} tetrahedra")
print(f"  - Demagnetization potential field as point data")
print(f"  - Solution range: [{phi.min():.6f}, {phi.max():.6f}]")
print(f"\nTo visualize:")
print(f"  - Open {output_filename} in ParaView")
print(f"  - Apply 'Clip' or 'Slice' filters to see internal structure")
print(f"  - Use 'Calculator' filter to compute magnetic field: -grad(demagnetization_potential)")
print(f"  - Color by 'demagnetization_potential' to see field distribution")

Solution exported to demag_potential_3d.vtu
File contains:
  - Mesh with 3665 nodes and 20679 tetrahedra
  - Demagnetization potential field as point data
  - Solution range: [-0.000000, 0.000000]

To visualize:
  - Open demag_potential_3d.vtu in ParaView
  - Apply 'Clip' or 'Slice' filters to see internal structure
  - Use 'Calculator' filter to compute magnetic field: -grad(demagnetization_potential)
  - Color by 'demagnetization_potential' to see field distribution


## Analysis and Post-Processing

The exported VTU file contains the complete 3D demagnetization potential solution. Key features to analyze:

1. **Magnetic dipole field**: The solution resembles a magnetic dipole field from the uniformly magnetized cube
2. **Boundary conditions**: Zero potential at the outer boundaries (far-field condition)
3. **Field continuity**: Smooth transition from the magnetic region to air
4. **Symmetry**: The field exhibits the expected symmetry from the uniform z-magnetization

### Post-processing in ParaView:
- **Magnetic field computation**: Use Calculator filter with `-grad(demagnetization_potential)` to get **H** = -∇φ
- **Cross-sections**: Apply Slice filter to view field in different planes
- **Isosurfaces**: Create Contour filter to show equipotential surfaces
- **Vector visualization**: Display magnetic field vectors using Glyph filter

### Physical Interpretation

- **Inside the cube**: The demagnetization field opposes the magnetization
- **Outside the cube**: The field resembles that of a magnetic dipole
- **At interfaces**: Field continuity is automatically satisfied by the finite element method

This type of calculation is fundamental in micromagnetics and magnetic field computation.

### Experiments
- Change the magnetization direction (e.g., m = (1,0,0) for x-direction)
- Modify the cube geometry or add multiple magnetic objects
- Implement non-uniform magnetization patterns
- Add magnetic permeability differences between regions